In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
SEED = 42
np.random.seed(SEED)

data = pd.read_csv('DATA_PROVINSI.csv')

df = pd.DataFrame(data)

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Menambahkan kolom lag (pergeseran)
df['jumlah_kecelakaan_lag'] = df['jumlah_kecelakaan'].shift(1)
df = df.dropna() # Menghapus baris dengan nilai NaN

In [3]:
# Memisahkan data latih dan data uji sesuai rasio (0.8 untuk 80:20)
train_size = int(0.7 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Memisahkan fitur dan target untuk data latih
X_train = train_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_train = train_data['jumlah_kecelakaan'].values

# Memisahkan fitur dan target untuk data uji
X_test = test_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_test = test_data['jumlah_kecelakaan'].values

In [4]:
# Define evaluation function
def evaluate(individual):
    C, epsilon = individual
    if C <= 0 or epsilon <= 0 :
        return float('inf'),
    svr = SVR(kernel='linear', C=C, epsilon=epsilon)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    return mape,
    
# Setup DEAP framework
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Parameter ranges
parameter_ranges = [(1, 10), (0.001, 1)]

# Register genetic operators
toolbox.register("attr_float", random.uniform)
toolbox.register("individual", tools.initCycle, creator.Individual, 
                 (lambda: random.uniform(*parameter_ranges[0]),
                  lambda: random.uniform(*parameter_ranges[1])), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxUniform, indpb=0.1)  # Menggunakan crossover uniform
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # Menggunakan mutasi flipbit
toolbox.register("select", tools.selTournament, tournsize=3) # Menggunakan seleksi turnamen
toolbox.register("evaluate", evaluate)

In [5]:
# Initialize population
population_size = 40
population = toolbox.population(n=population_size)

# Genetic algorithm parameters
crossover_rate = 0.6
mutation_rate = 0.4
num_generations = 200

# Run genetic algorithm
for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
        print(f"Individu: {ind}, MAPE: {fit[0]}")
    
    population = toolbox.select(offspring, k=len(population))

# Get the best individual
best_individual = tools.selBest(population, k=1)[0]
best_mape = best_individual.fitness.values[0]
C, epsilon = best_individual

# Test the best SVR model
best_svr = SVR(kernel='linear', C=C, epsilon=epsilon)
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print("\nParameter terbaik:")
print(f"C: {C}")
print(f"Epsilon: {epsilon}")
print(f"MAPE terbaik: {best_mape}")

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)

Generasi 1:
Individu: [6.571622037307934, 0.6127092175002502], MAPE: 24.428362554007556
Individu: [4.247942611331144, 0.6463759206992145], MAPE: 24.428362554007556
Individu: [2.3797231288711673, 0.21931319956771136], MAPE: 30.473314975956363
Individu: [5.07169803015868, 0.35246349948054856], MAPE: 27.749933439991064
Individu: [6.9282205380472455, 0.0], MAPE: inf
Individu: [3.7049024195088838, 0.5242029470941524], MAPE: 24.428362554007556
Individu: [8.224379128584992, 0.9191952722475152], MAPE: 24.428362554007556
Individu: [5.681661081461767, 0.8532333084893596], MAPE: 24.428362554007556
Individu: [1.175622687517392, 0.9911173938163771], MAPE: 24.428362554007556
Individu: [6.55611011267338, 0.650493501722468], MAPE: 24.428362554007556
Individu: [6.6372057697038205, 0.04150224967785556], MAPE: 32.481939939913445
Individu: [8.058934714839083, 0.076784790991808], MAPE: 34.272746398970376
Individu: [2.633358531602971, 0.9448985431245822], MAPE: 24.428362554007556
Individu: [2.73527930265274

In [6]:
# Nilai minimum dan maksimum asli dari data yang digunakan untuk normalisasi
data_min = 3  # Sesuaikan dengan nilai minimum data asli Anda
data_max = 36  # Sesuaikan dengan nilai maksimum data asli Anda

# Denormalisasi secara manual
y_pred_denormalized = y_pred * (data_max - data_min) + data_min

#pred_denormalized =  MinMaxScaler.inverse_transform(y_pred.reshape(1, -1))
print(y_pred_denormalized)

[18. 18. 18. 18. 18. 18. 18. 18. 18. 18. 18. 18. 18. 18. 18. 18. 18. 18.]
